In [126]:
from sage.all import *
reset()

In [127]:
p = 2
n = 2
q = p**n

# GR(4,2)
R  = PolynomialRing(Integers(4), 't'); t = R.gen()
GR = R.quotient(t**2+t+1, 'w'); w = GR.gen()
GR

Univariate Quotient Polynomial Ring in w over Ring of integers modulo 4 with modulus t^2 + t + 1

In [128]:
GR.cardinality()

16

In [129]:
list(GR)

[0,
 1,
 2,
 3,
 w,
 w + 1,
 w + 2,
 w + 3,
 2*w,
 2*w + 1,
 2*w + 2,
 2*w + 3,
 3*w,
 3*w + 1,
 3*w + 2,
 3*w + 3]

In [130]:
# def generalizedFrobenius(a, b):
#     return a**2 + GR(2)*b**2

def ringTrace(a, b):
    s = GR(0)
    for k in range(n):
        s += a**(2**k) + GR(2)*b**(2**k)
    return s 

for a in GR:
    for b in GR:
        if not ringTrace(a, b) == a + GR(2)*b + a**2 + GR(2)*b**2:
            raise Error('Trace not well defined!')
print('Trace defined!')

Trace defined!


In this case the ring generator $w$ coincides with the $\xi$.

In [144]:
w**3

1

In [131]:
T = [0] + [w**k for k in range((2**n - 2) + 1)]
T

[0, 1, w, 3*w + 3]

In [112]:
def brute2adic(x):
    for a in T:
        for b in T:
            if x == a + GR(2)*b:
                return (a,b)

In [113]:
mubs = [identity_matrix(q)]
for m in T:
    Bm = zero_matrix(SR, q, q)
    for j, v in enumerate(T):
        for i, w in enumerate(T):
            a, b = brute2adic(w**2 * m + 2 * w * v)
            pwr  = Integer(lift(ringTrace(a, b)))
            # print(pwr)
            Bm[i,j] = 1/sqrt(q) * I**pwr
            # Bm[i,j] = I**pwr
    mubs.append(Bm)

In [114]:
for i, m in enumerate(T):
    print(m)
    print(mubs[i+1])
    print('\n')

0
[ 1/2  1/2  1/2  1/2]
[ 1/2  1/2 -1/2 -1/2]
[ 1/2 -1/2 -1/2  1/2]
[ 1/2 -1/2  1/2 -1/2]


1
[   1/2    1/2    1/2    1/2]
[  -1/2   -1/2    1/2    1/2]
[-1/2*I  1/2*I  1/2*I -1/2*I]
[-1/2*I  1/2*I -1/2*I  1/2*I]


w
[   1/2    1/2    1/2    1/2]
[-1/2*I -1/2*I  1/2*I  1/2*I]
[  -1/2    1/2    1/2   -1/2]
[-1/2*I  1/2*I -1/2*I  1/2*I]


3*w + 3
[   1/2    1/2    1/2    1/2]
[-1/2*I -1/2*I  1/2*I  1/2*I]
[-1/2*I  1/2*I  1/2*I -1/2*I]
[  -1/2    1/2   -1/2    1/2]




In [115]:
def testMubs(mubs):
    for B in mubs:
        for V in mubs:
            if B != V:
                for j in range(4):
                    for k in range(4):
                        r = (transpose(B[:,j]) * V[:,k]).norm().abs()**2
                        if r != 1/4:
                            print(B)
                            print(V)
                            print(r)
                            raise Exception('Not MUBs!')
    return True

In [116]:
testMubs(mubs)

True

Now for five qubits, $\mathcal H = \mathbb C^{2^5}$.

In [145]:
reset()

p = 2
n = 5
q = p**n

In [146]:
F = GF(2**5, 'w', repr='poly'); w = F.gen()
F

Finite Field in w of size 2^5

In [147]:
charpoly(w) # defaults to Conway polynomial

x^5 + x^2 + 1

In [148]:
# GR(4,5)
R  = PolynomialRing(Integers(4), 't'); t = R.gen()

In [149]:
factor(t**5 + t**2 + 1)

t^5 + t^2 + 1

In [160]:
GR = R.quotient(t**5 + t**2 + 1, 'alpha'); alpha = GR.gen()
GR

Univariate Quotient Polynomial Ring in alpha over Ring of integers modulo 4 with modulus t^5 + t^2 + 1

In [161]:
alpha**5 + alpha**2 + 1

0

Ring generator $\alpha$ is not the multiplicative generator of the Teichmüller set as can be seen below:

In [162]:
alpha**(2**5-1)

2*alpha^3 + 2*alpha + 1

Andrés found the $2^5-1$-th primitive root of unity though:

In [163]:
xi = alpha**2 + 2*alpha + 1
xi**(2**5-1)

1

In [164]:
T = [0] + [xi**k for k in range(2**n - 2 + 1)]
len(T)

32

In [165]:
T[-1]*xi

1

In [166]:
T

[0,
 1,
 alpha^2 + 2*alpha + 1,
 alpha^4 + 2*alpha^2 + 1,
 3*alpha^4 + 3*alpha^3 + alpha^2 + alpha + 3,
 2*alpha^4 + 3*alpha^3 + alpha^2 + 2,
 alpha^4 + 3*alpha^3 + 2*alpha + 3,
 3*alpha^4 + 2*alpha^2 + 3*alpha + 2,
 alpha^4,
 alpha^4 + 3*alpha^3 + 2*alpha^2 + 3*alpha + 2,
 alpha^4 + alpha^3 + alpha^2 + 2*alpha + 1,
 3*alpha^2 + 3*alpha + 2,
 3*alpha^4 + alpha^3 + 3*alpha^2 + 3*alpha + 2,
 3*alpha^3 + 3,
 2*alpha^4 + 3*alpha^3 + 2*alpha,
 3*alpha^3 + alpha^2 + 1,
 3*alpha^4 + alpha^3 + 3*alpha^2 + 2*alpha + 2,
 2*alpha^3 + 2*alpha^2 + 3*alpha + 3,
 2*alpha^4 + alpha^3 + alpha^2 + alpha + 1,
 alpha^4 + 2*alpha^3 + 3*alpha^2 + alpha,
 alpha^2,
 alpha^4 + 2*alpha^3 + alpha^2,
 2*alpha^4 + 3*alpha^3 + alpha^2 + 3*alpha,
 alpha^4 + 2*alpha^3 + alpha + 1,
 alpha^4 + 2*alpha^3 + 3*alpha^2 + 2*alpha + 1,
 alpha^3 + 3*alpha + 1,
 2*alpha^4 + 2*alpha^2 + alpha,
 3*alpha^3 + 3*alpha,
 2*alpha^4 + 2*alpha^3 + 3*alpha^2 + 3*alpha + 1,
 alpha^4 + alpha^3 + 3*alpha + 3,
 3*alpha^4 + 3*alpha^3 + 2*alp

In [167]:
xi

alpha^2 + 2*alpha + 1